In [2]:
# from utils import *
import random
from Graph import Graph, Vertex
import numpy as np
import pandas as pd
import pickle
import os

In [6]:

# logic to use while inserting
records_dict = [{"id": i, "embed": list(row)} for i, row in enumerate(np.ones((3,3)))]
records_dict = [[x["id"]] + x["embed"] for x in records_dict]
print(records_dict)

for row in records_dict:
            dataKey = int(row[0])
            # print(dataKey)
            dataValue= np.array(row[1:],dtype=float)
            dataValue /= np.linalg.norm(dataValue)
            print(dataKey)
            print(dataValue)

[[0, 1.0, 1.0, 1.0], [1, 1.0, 1.0, 1.0], [2, 1.0, 1.0, 1.0]]
0
[0.57735027 0.57735027 0.57735027]
1
[0.57735027 0.57735027 0.57735027]
2
[0.57735027 0.57735027 0.57735027]


In [127]:
class VecDB():
    #TODO: 
    # 2 Constructors:
        # VecDB() followed by db.insert_records(records_dict)
        # VecDB(file_path , new_db = False) 
        # where file path is the path to the binary file having the index
        # retrieve(query, top_k)
    # L R 
    tenKparams=(15,17)
    hundredKparams=(20,20)
    
    
    # file path of binary file 
    def __init__(self, file_path='DBIndex/', new_db=True):
        # R= 17, L= 15, alpha = 2, K= 5,
        self.RecordsPerCluster=10**4
        if (new_db):
            #TODO: Check the records per cluster here or at insert records
            self.L,self.R = VecDB.tenKparams
        else: 
            self.L,self.R = VecDB.hundredKparams  
        self.alpha = 2
        self.offset = 0
        self.IndexPath = file_path
        self.DBGraph=None
        self.currentfile=0
        if(not new_db):
            # load graph from binary file
            #TODO: remove hard coded value
            self.DBGraph= pickle.load(open(file_path+"0.bin", "rb"))
    
    
    
    
    #TODO: CHECK after klam el mo3eed: might need to handle enk t4oof el directory 
    # records are list of dictionaries containing id and embeddings
    def insert_records(self, records):
        # might wanna change records per cluster if records are more than 100k
        print(len(records))
        if(len(records)>self.RecordsPerCluster):
            # split into clusters of 10k
            for i in range(0, len(records), self.RecordsPerCluster):
                temp = records[i:(i + self.RecordsPerCluster)]
                #TODO: check this line
                self.insert_records(temp)
                self.currentfile+=1
            return
        
        print("len in graph",len(records))
        self.DBGraph = self.Initialize_Random_Graph(records)
        self.Build_Index()
        
        # handle directory doesn't exist
        if not os.path.exists(self.IndexPath):
            os.makedirs(self.IndexPath)
        # set current file to len of current files in director
        self.currentfile+=len(os.listdir(self.IndexPath))
        print("writing to ",self.IndexPath+str(self.currentfile)+".bin")
        with open (self.IndexPath+str(self.currentfile)+".bin", 'wb') as f:
            pickle.dump(self.DBGraph, f)
    #TODO:  CHECK return to this later
    
    def load_binary_data(self, binary_file_path):
    # Load the data from the binary file
        with open(binary_file_path, 'rb') as f:
            data = pickle.load(f)
        # offset of last inserted record
        self.offset = data.iloc[0][0]
        # print(data)
        return data

    # Initialize a Graph connected randomly from the given records
    # records is a list of dictionaries containing id and embeddings
    # the function also calls the graph to calculate its medoid
    def Initialize_Random_Graph(self, records):
        DBGraph=Graph()
        DBdata = [[x["id"]] + x["embed"] for x in records]
        # print(len(DBdata))
        for row in DBdata:
            dataKey = int(row[0])
            dataValue= np.array(row[1:],dtype=float)
            # normalize 1 time initially
            dataValue /= np.linalg.norm(dataValue)
            DBGraph.add_vertex(Vertex(dataKey, dataValue))

        size= len(DBGraph.verticies)
        # for vertex in DBGraph:
        #     print(vertex)
        # print(size)
        # add to offset for when writing the next cluster of vertices
        if(size==0 or size==1):
            return
        # first element in records
        # self.offset=DBdata[0]["id"]
        #Building Edges
        for vertex in DBGraph:
            # we want R neighbors
            for i in range(self.R):
                # neighbor= DBGraph.get_vertex(int(random.random()*size) + self.offset)
                neighbor= DBGraph.get_vertex(int(random.random()*size))
                while(neighbor==vertex):
                    # neighbor= DBGraph.get_vertex(int(random.random()*size) + self.offset)
                    neighbor= DBGraph.get_vertex(int(random.random()*size))
                DBGraph.add_edge((vertex.key,vertex.value),(neighbor.key,neighbor.value))
        # get medoid of graph ie. calculate it if it's not calculated
        DBGraph.get_Medoid()

        return DBGraph

    def index_to_distance(self,id, query):
            vertex = self.DBGraph.get_vertex(id)
            dist = self.get_distance(vertex.value, query)
            return dist
    
    def retrieve(self,query,k):
        query /= np.linalg.norm(query)

        # top k from all clusters
        ClustersResults = []
        for idx, filename in enumerate(os.listdir(self.IndexPath)):
            filepath = os.path.join(self.IndexPath, filename)
            
            self.DBGraph = pickle.load(open(filepath,"rb"))

            TopK,_= self.Greedy_Search(self.DBGraph.medoid,query, k)
            # distance,index=((self.index_to_distance(k, query), k+(idx* self.offset)) for k in TopK)
            # ListToAdd=
            # add top k from this cluster to the list
            # print(ListToAdd)
            ClustersResults.extend([(self.index_to_distance(VertexId, query), VertexId) for VertexId in TopK])
            self.offset+=len(self.DBGraph.verticies)
            del self.DBGraph

        # print(ClustersResults)
        # sorts on first element of the tuple (which are the distances)
        ClustersResults.sort()
        # print(ClustersResults)
        # only get ids
        ClustersResults = [element[1] for element in ClustersResults[:k]]
        return ClustersResults
    
    # gets euclidean distance between 2 vectors
    def get_distance(self,v1,v2):
        dist = np.dot(v1,v2) #/ (np.linalg.norm(v1) * np.linalg.norm(v2))
        # print(v1.shape, v2.shape)
        return 1.0 - dist
        # return  np.linalg.norm(v1-v2)


    def get_min_dist_Key (self,AnyKeysSet,Query):
        # arrKey = list(AnyKeysSet)
        # arrEmb = np.array([self.DBGraph.get_vertex(i).value for i in arrKey])

        # a_norm = np.linalg.norm(arrEmb, axis=1)
        # b_norm = np.linalg.norm(Query)
        # dist = (arrEmb @ Query) / (a_norm * b_norm)

        # # minDist = np.linalg.norm(arrEmb, axis=1)
        # minIndex = np.argmin(dist)
        # return arrKey[minIndex], -1

        # put to 50 because the max distance is 2 anyway 
        min_dist=50
        min_vertex=None
        for vertexKey in AnyKeysSet:
            # print("vertex",vertex)
            # print("Query",Query[:3])
            vertex=self.DBGraph.get_vertex(vertexKey)
            dist=self.get_distance(vertex.value,Query)
            # print(dist)
            if(dist<min_dist):
                min_dist=dist
                min_vertex=vertex.key
        return min_vertex,min_dist


    #initially, start is the medoid
    # s is a vertex, Query is a vector
    # k is a number, L is a number
    # TODO: change them to be indices instead of vertices to save ram
    def Greedy_Search(self,start,Query, k):
        search_List={start.key}
        Visited=set()
        #TODO: make the visited and the possible frontier set of indices instead of vertices to save ram.
        possible_frontier=search_List
        Query /= np.linalg.norm(Query)
        while possible_frontier != set():
            # print('possible_frontier',possible_frontier)
            p_star,_= self.get_min_dist_Key(possible_frontier,Query)

            # print('pstar',p_star)
            # if p_star==None:
            #     # break
            #     print('frontier: ')
            #     for v in possible_frontier:
            #         print(v)
            #     print(possible_frontier==set())
            search_List=search_List.union(self.DBGraph.get_vertex(p_star).neighbors)
            Visited.add(p_star)
            if(len(search_List)>self.L):
                #update search list to retain closes L points to x_q
                search_ListL_L=list(search_List)
                search_ListL_L.sort(key=lambda x: self.get_distance(self.DBGraph.get_vertex(x).value,Query))
                # only maintain L closest points
                search_ListL_L=search_ListL_L[:self.L]
                search_List=set(search_ListL_L)

            possible_frontier=search_List.difference(Visited)

        search_ListL_L=list(search_List)
        search_ListL_L.sort(key=lambda x: self.get_distance(self.DBGraph.get_vertex(x).value,Query))
        # only maintain k closest points
        search_ListL_L=search_ListL_L[:k]
        search_List=set(search_ListL_L)
        # both are vectors of integers
        return search_List,Visited

    # # Robust pruning
    #candidate set is set of integers
    def Robust_Prune(self,point,candidate_set,alpha):
        # print(candidate_set)
        candidate_set=candidate_set.union(point.neighbors)
        # candidate_set.difference({point.key}) # changed
        candidate_set=candidate_set.difference({point.key}) # changed
        point.neighbors=set()
        # print("candidate_set", candidate_set)
        # while candidate_set not empty
        while candidate_set:
            p_star,_= self.get_min_dist_Key(candidate_set,point.value)
            point.neighbors.add(p_star)
            if(len(point.neighbors)==self.R):
                break
            DummySet=candidate_set.copy()
            for candidatePointKey in candidate_set:
                candidatePoint=self.DBGraph.get_vertex(candidatePointKey)
                # print(alpha * self.get_distance(self.DBGraph.get_vertex(p_star).value,candidatePoint.value), " <= ", self.get_distance(candidatePoint.value,point.value))
                # print(alpha)
                # ">=" condition is reversed as we use cosine similarity (paper uses L2), so higher value means it is closer
                if(alpha * self.get_distance(self.DBGraph.get_vertex(p_star).value,candidatePoint.value) <= self.get_distance(candidatePoint.value,point.value)):
                    # print("HEEEEEEEEEEEEEEEEELPPPPPPPPPPP !!!!!!!!!!!!!!!!!!!!!!")
                    DummySet.remove(candidatePoint.key)
            candidate_set=DummySet


    def Build_Index(self):

        # R = min(R, len(dataset))
        self.iterationOverGraph(1) #alpha=1
        self.iterationOverGraph(self.alpha) #alpha=2


    def iterationOverGraph(self,alpha):
        # print('medoid',medoid)
        randIndex = list(self.DBGraph.get_vertices())
        random.shuffle(randIndex)
        # random permutation + sequential graph update
        for n in randIndex:
            node = self.DBGraph.get_vertex(n)
            # print(n)

            (_,V) = self.Greedy_Search(self.DBGraph.medoid, node.value, 1) #K=1
            
            self.Robust_Prune(node, V, alpha)
            
            neighbors = node.get_neighbors()

            for inbKey in neighbors:

                # CHECK : The backward edge is always added
                # check here in case we shouldn't add it in all cases ? Might be incorrect?
                inb=self.DBGraph.get_vertex(inbKey)
                if len(inb.get_neighbors()) > self.R:
                    # print("inb.get_neighbors()", inb.get_neighbors())
                    U = inb.get_neighbors().union({node.key})
                    # inb.add_neighbor(node.key)
                    self.Robust_Prune(inb, U, alpha)
                else:
                    inb.add_neighbor(node.key)

# Main cell

In [61]:
R=12  # R > log n
k=10
L=10
alpha=2
path='./dataset/DB1m.bin'
step = 10_000
# Cluster files
with open(path, 'rb') as f:
    data = pickle.load(f)
    # print(data.shape)
    for i in range(0, data.shape[0],step):
        temp = data[i:(i + step)]
        temp[0] = temp[0] - i
        # print(temp.shape)
        # print(i,(i+step))
        # binarize and save
        pickle.dump(temp, open(f'./dataset/1Mbinaries/{i//step}.bin',"wb"))


C:\Users\BoodyBeeh\AppData\Local\Temp\ipykernel_28532\1102201325.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp[0] = temp[0]- i


In [64]:
R=18  # R > log n
k=10
L=20
alpha=2
path='./dataset/1Mbinaries/'
indexPath = './index/1Mindex/'
# Loop on clustered files
for idx, filename in enumerate(os.listdir(path)):
    filepath = os.path.join(path, filename)

    # print(filepath)
    # data = pickle.load(open(filepath,"rb"))
    # print(data)
    myDB= VecDB(path, R, L, alpha, k)
    myDB.Build_Index()
    # Write the index on disk
    pickle.dump(myDB, open(os.path.join(indexPath, str(idx)+".bin"),"wb"))
    del myDB

In [68]:
# # Write the index on disk
# pickle.dump(myDB, open('./index/DB1k_index.bin',"wb"))

# Load index
myDB = pickle.load(open('./index/1Mindex/0.bin',"rb"))

In [37]:
rng=np.random.default_rng(50)
vectors = rng.random((10**4, 70), dtype=np.float32)
db=VecDB()
records_dict = [{"id": i, "embed": list(row)} for i, row in enumerate(vectors)]
db.insert_records(records_dict)

In [120]:
random_vec=np.random.rand(1,70).T                    
actual_result_ids = np.argsort(vectors.dot(random_vec).T / (np.linalg.norm(vectors, axis=1) * np.linalg.norm(random_vec)), axis= 1).squeeze().tolist()[::-1]
TopK,_= db.retrieve(random_vec, 5)
# newDB=VecDB("DBIndex/0.bin",False)
# TopK,_= newDB.retrieve(random_vec, 5)
def evaluate_result(results):
    # scores are negative. So getting 0 is the best score.
    scores = []
    
    # case for retireving number not equal to top_k, socre will be the lowest
    if len(set(results)) != k or len(results) != k:
        # scores.append( -1 * len(results) * k)
        score= -1 * len(results) * k
        return score
    score = 0
    for id in results:
        try:
            # print(id)
            ind = actual_result_ids.index(id)
            if ind > k * 3:
                score -= 1
        except:
            # score -= len(results)
            score -= 1
    scores.append(score)

    return sum(scores) / len(scores)

print('------------------------')
print("topK", TopK)
print("Actual", actual_result_ids)
print(evaluate_result(TopK))

[<generator object VecDB.retrieve.<locals>.<genexpr> at 0x0000016C9FB009A0>]
[<generator object VecDB.retrieve.<locals>.<genexpr> at 0x0000016C9FB009A0>]


TypeError: 'generator' object is not subscriptable

In [128]:
rng=np.random.default_rng(50)
vectors = rng.random((10**4, 70), dtype=np.float32)
del vectors
vectors = rng.random((2* 10**4, 70), dtype=np.float32)
records_dict = [{"id": i, "embed": list(row)} for i, row in enumerate(vectors)]

newDB=VecDB("DBIndex/",False)
newDB.insert_records(records_dict)

writing to  DBIndex/1.bin


AttributeError: 'NoneType' object has no attribute 'key'

In [119]:
# mygraph=Graph()
# with open ("DBIndex/0.bin","rb")as f:
# # with open ("DBIndex/0.bin","wb")as f:
#     mygraph=pickle.load(f)
# for vertex in mygraph:
#     print(vertex)
# rng=np.random.default_rng()
vec=np.random.rand(1,70).T                    
actual_result_ids = np.argsort(vectors.dot(vec).T / (np.linalg.norm(vectors, axis=1) * np.linalg.norm(vec)), axis= 1).squeeze().tolist()[::-1]
newDB=VecDB("DBIndex/",False)
TopK= newDB.retrieve(vec, 5)
k=5
# newDB=VecDB("DBIndex/0.bin",False)
# TopK,_= newDB.retrieve(random_vec, 5)
def evaluate_result(results):
    # scores are negative. So getting 0 is the best score.
    scores = []
    
    # case for retireving number not equal to top_k, socre will be the lowest
    if len(set(results)) != k or len(results) != k:
        # scores.append( -1 * len(results) * k)
        score= -1 * len(results) * k
        return score
    score = 0
    for id in results:
        try:
            # print(id)
            ind = actual_result_ids.index(id)
            if ind > k * 3:
                score -= 1
        except:
            # score -= len(results)
            score -= 1
    scores.append(score)

    return sum(scores) / len(scores)

print('------------------------')
print("topK", TopK)
print("Actual", actual_result_ids)
print(evaluate_result(TopK))

------------------------
topK [6367, 6714, 5480, 4162, 3505]
Actual [1359, 6802, 6367, 5445, 6714, 5480, 4764, 4162, 9273, 5187, 3505, 3556, 1065, 8965, 8299, 5738, 7531, 1997, 5168, 3106, 11, 5862, 7430, 8684, 8982, 8227, 3808, 7169, 7902, 5092, 363, 6041, 7951, 2202, 5789, 6870, 5497, 3818, 3462, 4749, 5905, 6027, 3130, 2795, 1541, 2463, 601, 3666, 2724, 5755, 8717, 570, 7275, 8641, 8794, 9976, 1457, 921, 6323, 620, 391, 6506, 1242, 3868, 3479, 9644, 6057, 8445, 9995, 6746, 8566, 5821, 8888, 7493, 5424, 7510, 2120, 5586, 5922, 9803, 5992, 9984, 6747, 9263, 8260, 2375, 4844, 2358, 2996, 8061, 454, 7608, 8678, 8675, 6762, 8421, 5995, 522, 2437, 2439, 3827, 1539, 5872, 7361, 1034, 5649, 431, 866, 8718, 1793, 3753, 929, 6313, 5831, 8178, 6472, 9830, 7935, 9238, 9732, 1460, 3217, 6283, 9534, 6457, 1720, 906, 2636, 6713, 5070, 1610, 24, 1090, 3801, 9316, 5160, 6287, 2006, 5443, 7918, 9568, 6486, 9611, 3950, 6942, 3331, 1343, 8868, 5363, 1632, 3177, 3911, 5931, 6194, 3908, 3521, 7624, 5047,

# Recall

In [159]:

k = 5
random_vec=np.random.rand(1,70).T

TopK,_= myDB.Greedy_Search(myDB.medoid,random_vec, k)

# actual eqn to be tested with
vectors=np.array([myDB.DBGraph.get_vertex(i).value for i in myDB.DBGraph.get_vertices()])
print(vectors.shape)

actual_result_ids = np.argsort(vectors.dot(random_vec).T / (np.linalg.norm(vectors, axis=1) * np.linalg.norm(random_vec)), axis= 1).squeeze().tolist()[::-1]

# print('The vector is:')
# print(random_vec)
print('------------------------')
print('len of TopK is:',len(TopK))

# print(TopK.shape)
# for lol in TopK:
    # print(lol)
    # print(myDB.get_distance(myDB.DBGraph.get_vertex(lol).value,random_vec))
    # print('------------------------')


def evaluate_result(results):
    # scores are negative. So getting 0 is the best score.
    scores = []
    
    # case for retireving number not equal to top_k, socre will be the lowest
    if len(set(results)) != k or len(results) != k:
        # scores.append( -1 * len(results) * k)
        score= -1 * len(results) * k
        return score
    score = 0
    for id in results:
        try:
            # print(id)
            ind = actual_result_ids.index(id)
            if ind > k * 3:
                score -= 1
        except:
            # score -= len(results)
            score -= 1
    scores.append(score)

    return sum(scores) / len(scores)

print('------------------------')
print("topK", TopK)
print("Actual", actual_result_ids)
print(evaluate_result(TopK))

(100000, 70)
------------------------
len of TopK is: 5
------------------------
topK {93153, 17129, 15242, 32438, 53111}
Actual [74579, 64113, 86999, 36325, 49787, 86192, 40626, 17625, 94655, 64995, 55195, 77056, 49387, 3107, 85901, 5625, 53111, 33116, 22118, 83206, 24358, 37305, 28768, 84462, 51365, 93747, 58347, 52458, 4737, 29184, 85381, 89254, 63820, 58731, 70300, 29045, 91910, 93499, 192, 71013, 65380, 59201, 56004, 24036, 17879, 82099, 69568, 51548, 89527, 88206, 46458, 93153, 60941, 65204, 46516, 17129, 93535, 81605, 67814, 54143, 91385, 28877, 88017, 29048, 55569, 11389, 55905, 27739, 64419, 35190, 22940, 14185, 70005, 37632, 72819, 63708, 88825, 45417, 51846, 87146, 34457, 58569, 72910, 83288, 72062, 61977, 5573, 70354, 52569, 62635, 76039, 6669, 56584, 57492, 56220, 527, 2001, 92558, 18433, 99653, 30707, 54671, 98471, 84395, 14675, 29998, 90434, 95728, 31958, 34606, 26723, 95825, 71551, 66344, 96575, 35838, 27000, 23624, 54473, 32503, 3402, 87506, 94716, 3266, 51985, 59472, 